# EDGAR MD&A Extraction - Colab Setup

This notebook sets up your Google Colab environment for large-scale MD&A extraction from SEC filings.

## What This Does:
1. Mounts Google Drive
2. Installs required dependencies
3. Clones the edgar-crawler repository
4. Creates necessary directory structure
5. **Configures SEC User-Agent (REQUIRED)**
6. Sets up WRDS authentication

## Before You Start:
- ✅ Have a WRDS account with COMPUSTAT access
- ✅ Have sufficient Google Drive storage (~200-300 GB recommended)
- ✅ Have Colab Pro (optional but recommended for 24-hour sessions)
- ✅ **Have your name and email ready for SEC identification**

---

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set up project directory
PROJECT_DIR = '/content/drive/MyDrive/EDGAR_Project'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

print(f"✅ Google Drive mounted")
print(f"✅ Project directory: {PROJECT_DIR}")

## Step 2: Install Dependencies

In [ ]:
# Install WRDS library
!pip install -q wrds

# Install compatible versions to avoid conflicts with Colab's pre-installed packages
# Note: Colab's 'datasets' package requires dill<0.3.9 and multiprocess<0.70.17
!pip install -q 'dill<0.3.9' 'multiprocess<0.70.17'

# Install pathos dependencies (pox, ppft) first
!pip install -q pox ppft

# Install pathos WITHOUT upgrading dependencies (--no-deps)
# This prevents it from upgrading dill and multiprocess to incompatible versions
!pip install -q --no-deps pathos

# Install other required packages
!pip install -q beautifulsoup4 lxml requests pandas tqdm

print("✅ All dependencies installed")

## Step 3: Clone/Update EDGAR Crawler Repository

In [ ]:
import os

REPO_DIR = os.path.join(PROJECT_DIR, 'edgar-crawler')

if not os.path.exists(REPO_DIR):
    print("Cloning edgar-crawler repository...")
    # Clone from haowenluo fork with WRDS workflow scripts
    # Using the feature branch that contains the WRDS integration
    !git clone -b claude/add-download-functionality-011CUuZUGtr16XsT2Ann77oP https://github.com/haowenluo/edgar-crawler.git
    print("✅ Repository cloned")
else:
    print("Repository already exists. Pulling latest changes...")
    %cd {REPO_DIR}
    !git pull
    print("✅ Repository updated")

# Change to repo directory
%cd {REPO_DIR}
print(f"\n📁 Working directory: {os.getcwd()}")

## Step 4: Create Directory Structure

In [ ]:
# Create necessary directories
directories = [
    'datasets/RAW_FILINGS/10-K',
    'datasets/EXTRACTED_FILINGS',
    'datasets/INDICES',
    'extraction_configs',
    'logs',
    'wrds_data'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Created: {directory}")

print("\n✅ All directories created")

## Step 4.5: Configure SEC User-Agent (REQUIRED)

⚠️ **IMPORTANT:** The SEC requires you to identify yourself when downloading filings.

Per the [SEC's fair access policy](https://www.sec.gov/os/accessing-edgar-data), you must provide:
- Your real name
- Your real email address

This allows the SEC to contact you if there are issues with your requests.

**You will get 403 errors if you skip this step!**

In [ ]:
import json

# Update config.json with your information
# Replace with YOUR name and email
YOUR_NAME = input("Enter your name: ")
YOUR_EMAIL = input("Enter your email: ")

user_agent = f"{YOUR_NAME} {YOUR_EMAIL}"

# Update config file
with open('config.json', 'r') as f:
    config = json.load(f)

config['download_filings']['user_agent'] = user_agent

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n✅ User-Agent configured: {user_agent}")
print("\nThis will be used for all SEC requests.")
print("The SEC may contact you at this email if there are issues.")

## Step 5: WRDS Authentication

You'll need to authenticate with WRDS. You have two options:

### Option A: Interactive (Prompts for password each session)

In [ ]:
import wrds
from getpass import getpass

# Enter your WRDS username
wrds_username = input("Enter your WRDS username: ")

# Test connection
print("\nConnecting to WRDS...")
try:
    db = wrds.Connection(wrds_username=wrds_username)
    print("✅ Successfully connected to WRDS!")
    db.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nPlease check your credentials and try again.")

### Option B: Store Credentials (Using Colab Secrets - Recommended)

To avoid re-entering credentials:
1. Click the 🔑 key icon in the left sidebar (Secrets)
2. Add a new secret:
   - Name: `WRDS_USERNAME`
   - Value: Your WRDS username
3. Add another secret:
   - Name: `WRDS_PASSWORD`
   - Value: Your WRDS password
4. Toggle "Notebook access" ON for both secrets
5. Run the cell below

In [ ]:
# Uncomment and run this if you've set up Colab Secrets

# from google.colab import userdata
# import os

# os.environ['WRDS_USERNAME'] = userdata.get('WRDS_USERNAME')
# os.environ['WRDS_PASSWORD'] = userdata.get('WRDS_PASSWORD')

# print("✅ WRDS credentials loaded from secrets")

## Step 6: Verify Setup

In [ ]:
import os
import sys

print("="*50)
print("SETUP VERIFICATION")
print("="*50)

# Check working directory
print(f"\n📁 Working Directory: {os.getcwd()}")

# Check if key files exist
key_files = [
    'download_filings.py',
    'extract_items.py',
    'wrds_downloader.py',
    'colab_batch_downloader.py',
    'flexible_extractor.py'
]

print("\n📄 Key Files:")
for file in key_files:
    exists = "✅" if os.path.exists(file) else "❌"
    print(f"  {exists} {file}")

# Check directories
print("\n📂 Directories:")
for directory in directories:
    exists = "✅" if os.path.exists(directory) else "❌"
    print(f"  {exists} {directory}")

# Check Python version
print(f"\n🐍 Python Version: {sys.version.split()[0]}")

# Check available disk space
import shutil
total, used, free = shutil.disk_usage("/content/drive")
print(f"\n💾 Google Drive Space:")
print(f"   Total: {total // (2**30)} GB")
print(f"   Used: {used // (2**30)} GB")
print(f"   Free: {free // (2**30)} GB")

if free < 200 * (2**30):  # Less than 200 GB
    print("   ⚠️  Warning: You may need more space for large downloads")
else:
    print("   ✅ Sufficient space available")

print("\n" + "="*50)
print("✅ SETUP COMPLETE!")
print("="*50)
print("\nNext steps:")
print("1. Review COLAB_GUIDE.md for detailed workflow")
print("2. Configure your sample in wrds_config.py")
print("3. Run wrds_downloader.py to get firm identifiers")
print("4. Run colab_batch_downloader.py to download filings")
print("5. Run flexible_extractor.py to extract MD&A sections")

## Optional: Keep-Alive Script

This prevents Colab from disconnecting due to inactivity (run in a separate cell and keep it running)

In [ ]:
# Keep-alive script (optional)
# This simulates activity to prevent auto-disconnect

from IPython.display import display, Javascript

display(Javascript('''
function KeepClicking(){
    console.log("Keeping session alive...");
    document.querySelector("colab-connect-button").click();
}
setInterval(KeepClicking, 60000);  // Click every 60 seconds
'''))

print("✅ Keep-alive script activated")
print("Note: This helps but doesn't guarantee no disconnection.")
print("Make sure to enable resume capability in the download scripts.")

---

## You're All Set! 🎉

Your environment is now configured. Proceed to the main workflow by following the **COLAB_GUIDE.md** documentation.

### Quick Start:

1. **Download firm identifiers from WRDS:**
   ```python
   !python wrds_downloader.py
   ```

2. **Download SEC filings (takes ~1 week):**
   ```python
   !python colab_batch_downloader.py --batch-size 500
   ```

3. **Extract MD&A sections:**
   ```python
   !python flexible_extractor.py --config extraction_configs/mda_only.json
   ```

4. **Generate consolidated CSV:**
   ```python
   !python consolidate_output.py --item 7 --output mda_analysis.csv
   ```

See **COLAB_GUIDE.md** for detailed instructions!